# neo4j graph fun

# Preliminaries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from pprint import pprint
import neo4j

In [3]:
import hashlib
from hashlib import sha256

In [4]:
from functools import lru_cache as cache
from collections import defaultdict

In [5]:
import ipywidgets as widgets
from sidecar import Sidecar
from nnvis import NetResponsePlot

In [6]:
from graph_utils_neo4j import NumpyStore
from nnbench import Thing

## Connecting

Need to get the `gpu-jupyter` and the `neo4j` docker containers connected. If run bare, something like:

    docker network connect gpu-jupyter_default gpu-jupyter 
    docker network connect gpu-jupyter_default neo4j
    docker network inspect gpu-jupyter_default 
    
Docker has better ways than this.

In [7]:
driver = neo4j.GraphDatabase.driver("neo4j://172.19.0.2:7687", auth=("neo4j", "test"))

### Alive?

In [8]:
driver.verify_connectivity()

<ipython-input-8-b734c674563a>:1: ExperimentalWarning: The configuration may change in the future.
  driver.verify_connectivity()


{IPv4Address(('172.19.0.2', 7687)): [{'ttl': 300,
   'servers': [{'addresses': ['172.19.0.2:7687'], 'role': 'WRITE'},
    {'addresses': ['172.19.0.2:7687'], 'role': 'READ'},
    {'addresses': ['172.19.0.2:7687'], 'role': 'ROUTE'}]}]}

# The Model
    Investigation -> Experiment -> multiple ResultDAGs
`ResultDAG` is

    (netState, params)-[mutation]->(netState, params)-[mutation ...
                     +-[mutation]->(netstate, params) ...
etc. `mutation` can be a learning trajectory, or an edit.

Perhaps `mutation` can be expressed in python.

Generally the results of experiments are preferred to be reproducible, but they won't always be, when they import entropy.

## Some neural nets

In [9]:
from nn import Network, Layer, IdentityLayer, AffineLayer, MapLayer
from nnbench import NetMaker, NNMEG

In [10]:
mnm = NetMaker(NNMEG)
xor_net = mnm('2x2tx1t')
adc_net = mnm('1x8tx8tx3t')

## ... and training data

In [11]:
xor_training_batch = (np.array([[-0.5, -0.5],
                            [-0.5,  0.5],
                            [ 0.5,  0.5],
                            [ 0.5, -0.5]]),
                  np.array([[-0.5],
                            [ 0.5],
                            [-0.5],
                            [ 0.5]]))

In [12]:
def adc(input):
    m = max(0, min(7, int(8*input)))
    return np.array([(m>>2)&1, (m>>1)&1, m&1]) * 2 - 1

vadc = lambda v: np.array([adc(p) for p in v])
#plot_ADC(vadc)

In [13]:
x = np.arange(0, 1, 1.0/(8*1)).reshape(-1,1) # 1 point in each output region
adc_training_batch = (x, vadc(x))

In [14]:
def data_from_run():
    def get_sequence(tx, rv):
        for record in tx.run("MATCH p=(head:net)-[:LEARNED*]->(tail:net) "
                             "WHERE NOT ()-[:LEARNED]->(head) "
                             "AND NOT (tail)-[:LEARNED]->() "
                             "RETURN "
                             "head, "
                             "[x IN nodes(p) | x.ksv] as ksvs, "
                             "[x IN nodes(p) | x.loss] as losses "
                            ):
            rv.head = record['head']
            rv.ksvs = record['ksvs']
            rv.losses = record['losses']

    rv = Thing
    with driver.session() as session:
        session.read_transaction(get_sequence, rv)
    return rv

In [15]:
def data_from_run():
    def get_sequence(tx, rv):
        for record in tx.run("MATCH p=(head:net)-[:LEARNED*]->(tail:net) "
                             "WHERE NOT ()-[:LEARNED]->(head) "
                             "AND NOT (tail)-[:LEARNED]->() "
                             "RETURN "
                             "head, "
                             "[x IN nodes(p) | x.ksv] as ksvs, "
                             "[x IN nodes(p) | x.loss] as losses "
                            ):
            head_ksv = record['head']['ksv']
            t = rv[head_ksv]
            t.head = record['head']
            t.ksvs = record['ksvs']
            t.losses = record['losses']

    rv = defaultdict(Thing)
    with driver.session() as session:
        session.read_transaction(get_sequence, rv)
    return rv

In [16]:
nps = NumpyStore(driver)

@cache
def sv_from_ksv(ksv):
    return nps[ksv]

In [17]:
def show_adcs_n(n):
    for nrp, thing in zip(nrps, things):
        nrp(sv_from_ksv(thing.ksvs[n]))
    return [thing.losses[n] for thing in things]

In [34]:
run_data = data_from_run()

In [35]:
hks, things = (v for v in zip(*run_data.items()))
nets = [mnm(t.head['shorthand']) for t in things]
nrps = [NetResponsePlot(net, height='220px', margin=30, title=net.shorthand) for net in nets]

In [36]:
grid = widgets.GridspecLayout(5, 2,
                      grid_gap='10px',
                      justify_content='center',
                      align_items='top')

nrps = [NetResponsePlot(net, height='220px', margin=30, title=net.shorthand) for net in nets]

for i, nrp in enumerate(nrps):
    column = i % grid.n_columns
    row = i // grid.n_columns
    grid[row, column] = nrp.fig

frame_w = widgets.IntSlider(min=0,
                            max=max(len(t.ksvs) for t in things)-1,
                            step=1, value=0)

grid[-1,-1] = widgets.VBox((frame_w,))
        
with Sidecar(title='grid') as gside:
    display(grid)
    
widgets.interact(show_adcs_n, n=frame_w)

interactive(children=(IntSlider(value=0, description='n', max=68), Output()), _dom_classes=('widget-interact',…

<function __main__.show_adcs_n(n)>

___

# Scrapyard

In [21]:
assert False, "Scrapyard below"

AssertionError: Scrapyard below

# Try Queries

In [ ]:
def get_sequence(tx):
    for record in tx.run("MATCH p=(head:net)-[:LEARNED*]->(tail:net)"
                         "WHERE NOT ()-[:LEARNED]->(head)"
                         "AND NOT (tail)-[:LEARNED]->()"
                         "RETURN p, [x IN nodes(p) | x.ksv] as ksvs"):
        print(record['ksvs'])
        p = record['p']
        print(type(p))
        print(record['ksvs'])
        for r in p.relationships:
            print(r.start_node['ksv'], r['ts'], r.end_node['ksv'])

with driver.session() as session:
    session.read_transaction(get_sequence)

In [ ]:
def get_sequence(tx):
    for record in tx.run("MATCH p=(head:net)-[:LEARNED*]->(tail:net) "
                         "WHERE NOT ()-[:LEARNED]->(head) "
                         "AND NOT (tail)-[:LEARNED]->() "
                         "RETURN "
                         "head, "
                         "[x IN nodes(p) | x.ksv] as ksvs, "
                         "[x IN nodes(p) | x.loss] as losses "
                        ):
        head = record['head']
        ksvs = record['ksvs']
        losses = record['losses']
        print(ksvs, losses)
        print(head)
        
with driver.session() as session:
    session.read_transaction(get_sequence)

In [ ]:
d = defaultdict(Thing)

In [ ]:
d['a'].foo = 3
d['b'].bar = lambda x: x*x

In [ ]:
d

In [ ]:
d['b'].bar(3)